In [10]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [11]:
# Define the self-attention layer
class SelfAttention(layers.Layer):
    def __init__(self):
        super(SelfAttention, self).__init__()
        self.gamma = tf.Variable(initial_value=tf.zeros((1,), dtype='float32'), trainable=True)

    def build(self, input_shape):
        _, height, width, channels = input_shape
        self.query_conv = layers.Conv2D(channels // 8, kernel_size=1, use_bias=False)
        self.key_conv = layers.Conv2D(channels // 8, kernel_size=1, use_bias=False)
        self.value_conv = layers.Conv2D(channels, kernel_size=1, use_bias=False)

    def call(self, inputs):
        # Compute the query, key, and value matrices
        query = self.query_conv(inputs)
        key = self.key_conv(inputs)
        value = self.value_conv(inputs)

        # Compute the dot-product attention
        attention_logits = tf.matmul(query, key, transpose_b=True)
        attention_weights = tf.nn.softmax(attention_logits)
        attention_output = tf.matmul(attention_weights, value)

        # Apply the gamma scaling factor
        attention_output = self.gamma * attention_output

        # Add the attention output to the original input
        output = inputs + attention_output
        return output




In [12]:
# Define the CNN with one self-attention layer
def cnn_sa():
    input_shape = (32, 32, 3)
    num_classes = 10

    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        SelfAttention(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        SelfAttention(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

In [13]:
# loading cifar 10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert the labels to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = cnn_sa()

# Compile and training the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')

Epoch 1/10
782/782 [==============================] - 417s 529ms/step - loss: 1.4350 - accuracy: 0.5115 - val_loss: 0.9623 - val_accuracy: 0.6647
Epoch 2/10
782/782 [==============================] - 407s 521ms/step - loss: 0.9281 - accuracy: 0.6775 - val_loss: 0.8805 - val_accuracy: 0.6942
Epoch 3/10
782/782 [==============================] - 405s 518ms/step - loss: 0.7825 - accuracy: 0.7299 - val_loss: 0.8013 - val_accuracy: 0.7204
Epoch 4/10
782/782 [==============================] - 400s 512ms/step - loss: 0.6898 - accuracy: 0.7621 - val_loss: 0.7198 - val_accuracy: 0.7504
Epoch 5/10
782/782 [==============================] - 403s 515ms/step - loss: 0.6172 - accuracy: 0.7879 - val_loss: 0.6879 - val_accuracy: 0.7597
Epoch 6/10
782/782 [==============================] - 408s 522ms/step - loss: 0.5526 - accuracy: 0.8096 - val_loss: 0.6456 - val_accuracy: 0.7778
Epoch 7/10
782/782 [==============================] - 402s 514ms/step - loss: 0.4996 - accuracy: 0.8268 - val_loss: 0.6407 -